In [ ]:
# Copyright 2024 Rittman Analytics ltd
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# P&L Query Self-Learning Chatbot Prototype with RAG and SQL Agent

## Overview

This Jupyter notebook implements an advanced question-answering system for Profit and Loss (P&L) data analysis. It combines Retrieval-Augmented Generation (RAG), vector store queries, and SQL-based data retrieval to provide comprehensive and accurate answers to financial questions.

## Key Features

1. Hybrid RAG and SQL-based querying
2. Dynamic decision-making between RAG and SQL approaches
3. Feedback loop for continuous improvement
4. Vector store for storing and retrieving successful Q&A pairs
5. Flexible time period handling for financial queries

## System Architecture

### 1. Vector Stores

The system uses two vector stores:
- `vector_store`: Stores pre-created P&L report analyses
- `qa_vector_store`: Stores successful question-answer pairs

Both vector stores are implemented using BigQuery and the `BigQueryVectorStore` class from LangChain.

### 2. Embedding Model

We use the `VertexAIEmbeddings` model to generate embeddings for both the pre-created analyses and the Q&A pairs.

### 3. Language Model

The system uses OpenAI's GPT-4 model (via `ChatOpenAI`) for natural language understanding and generation.

### 4. SQL Database

The P&L data is stored in BigQuery tables, accessed through SQLAlchemy.

## Workflow

1. **Question Input**: The user inputs a question about P&L data.

2. **RAG vs SQL Decision**:
   - The `should_query_vector_store` function determines whether to use RAG or SQL-based querying.
   - It considers the question content, time frame, and available data in the vector store.

3. **RAG Approach** (if chosen):
   - The system queries the `vector_store` for relevant pre-created analyses.
   - It uses the `similarity_search` method to find the most relevant document.
   - The retrieved content is summarized and presented as the answer.

4. **SQL Approach** (if chosen or if RAG fails):
   - The system extracts relevant time periods and financial categories from the question.
   - It constructs a SQL query using the `SQLDatabaseToolkit` and `create_sql_agent` from LangChain.
   - The query is executed against the BigQuery database, and the results are processed to form an answer.

5. **Answer Evaluation**:
   - The `evaluate_answer_relevance` function uses the language model to assess the relevance and quality of the generated answer.

6. **User Feedback**:
   - The system asks the user if the answer is sufficient.
   - If not, it requests feedback on how to improve the answer.

7. **Question Refinement**:
   - If the user is not satisfied, the system uses the language model to generate an improved question based on the feedback.
   - The process repeats with the refined question (up to a maximum of 3 iterations).

8. **Storing Successful Q&A Pairs**:
   - When a user is satisfied with an answer, the Q&A pair is stored in the `qa_vector_store`.
   - This stored pair can be used to inform future similar questions.

## Key Components

### 1. `extract_time_periods` Function
Identifies and extracts various time periods (years, months, quarters) mentioned in the question.

### 2. `construct_date_filter` Function
Constructs SQL date filters based on the extracted time periods.

### 3. `ask_question` Function
The core function that orchestrates the question-answering process, including the decision between RAG and SQL approaches.

### 4. `ask_question_with_feedback_and_learning` Function
Implements the feedback loop and question refinement process.

### 5. `store_successful_qa` Function
Stores successful Q&A pairs in the vector store for future reference.

## Continuous Learning

The system improves over time through two mechanisms:
1. Storing successful Q&A pairs in the vector store, which can be retrieved for similar future questions.
2. The feedback loop, which allows for question refinement based on user input.

## Usage

1. Set up the required Google Cloud and OpenAI credentials.
2. Run the notebook cells in order to initialize all components.
3. Use the `main` function to start an interactive Q&A session about P&L data.

## Customization

The system can be customized by:
- Modifying the `vector_store_content_description` to match specific P&L data structures.
- Adjusting the SQL views and table names in the `determine_view` function.
- Expanding the `extract_time_periods` function to handle more complex date formats.

In [ ]:
pip install bs4 langchain langchain-community langchain-google-community langchain_google_vertexai langchain-openai openai chromadb tiktoken tabulate sqlalchemy sqlalchemy-bigquery google-cloud-bigquery

In [21]:
import os
from google.cloud import bigquery
from google.api_core import exceptions
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain, LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd
from collections import defaultdict
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore
import re
import uuid
from datetime import datetime
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from langchain.docstore.document import Document


In [ ]:
service_account_file = "/content/ra-development-bf14f75f80cd.json"
project = "ra-development"
dataset = "analytics_finance_demo"
location = "europe-west2"
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'
os.environ["OPENAI_API_KEY"] = "<YOUR_OPEN_AI_API_KEY>"

vector_store_content_description = """
The vector store contains pre-created analysis with the following information:
1. Key Metrics
   - Revenue, Overheads, Cost of Delivery, Gross & Net Profit, Retained Earnings net amounts
   - Account Sub-Category and Account Group net amounts
2. Key Metrics Calculation:
   - Month-over-month, year-to-date, and budget variance calculations
   - Gross margin percentages
3. Significant Transaction Identification:
   - Transactions exceeding a certain percentage threshold of the total account group amount
   - Cancelling transactions are identified and excluded
   - Context for each significant transaction (new or changed from previous month)
4. Overhead Trend Analysis:
   - Monthly growth rates for overhead categories over the last 6 months
   - Account groups with significant average monthly growth (> 10%)
5. Identification of New Repeating (Recurring) Transactions:
   - Transactions with the same description appearing consistently over the last 3 months

The analysis covers the last three months and the current year-to-date at summary level for category, subcategory, and account group levels.
"""

# Create a BigQuery client
client = bigquery.Client.from_service_account_json(service_account_file)

# Initialize embedding model
embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest",
    project=project
)

# Initialize BigQueryVectorStore containing P&L report analysis texts
vector_store = BigQueryVectorStore(
    project_id=project,
    dataset_name=dataset,
    table_name="pl_reports_vector_storage",
    location=location,
    embedding=embedding_model,
)

def create_successful_qa_table():
    client = bigquery.Client()
    table_id = f"{project}.{dataset}.successful_qa_pairs"

    schema = [
        bigquery.SchemaField("text", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("embedding", "FLOAT64", mode="REPEATED"),
        bigquery.SchemaField("id", "STRING", mode="REQUIRED"),
    ]

    table = bigquery.Table(table_id, schema=schema)
    table.clustering_fields = ["id"]

    try:
        table = client.create_table(table)
        print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")
    except exceptions.Conflict:
        print(f"Table {table_id} already exists.")
    except Exception as e:
        print(f"An error occurred while creating the table: {str(e)}")

# Initialize a new vector store for storing successful Q&A pairs
def initialize_qa_vector_store():
    create_successful_qa_table()
    return BigQueryVectorStore(
        project_id=project,
        dataset_name=dataset,
        table_name="successful_qa_pairs",
        location=location,
        embedding=embedding_model,
    )

qa_vector_store = initialize_qa_vector_store()

def load_vector_storage():
    query = f"""
    SELECT date_month as month, report_analysis, invoice_analysis, recurring_payments_analysis
    FROM `{project}.{dataset}.pl_reports`
    """
    df = client.query(query).to_dataframe()

    for _, row in df.iterrows():
        month = row['month']
        for analysis_type in ['report_analysis', 'invoice_analysis', 'recurring_payments_analysis']:
            text = row[analysis_type]
            metadata = {
                'month': month,
                'analysis_type': analysis_type
            }
            vector_store.add_texts([text], metadatas=[metadata])

    print("Vector storage loaded successfully.")

def get_available_months():
    query = f"""
    SELECT DISTINCT month as month
    FROM `{project}.{dataset}.pl_reports_vector_storage`
    ORDER BY month DESC
    """
    df = client.query(query).to_dataframe()
    return df['month'].tolist()

# Add this function to determine the valid time range
def get_valid_time_range(available_months):
    if not available_months:
        return None, None

    latest_month = max(available_months)
    earliest_month = min(available_months)

    # Calculate the start of the year for the latest month
    year_start = latest_month.replace(month=1, day=1)

    # The valid range includes two months before the earliest available month
    valid_start = (earliest_month - relativedelta(months=2)).replace(day=1)

    return valid_start, latest_month

def create_lookup_tables():
    query = f"""
    SELECT DISTINCT
        account_report_group,
        account_report_sub_category,
        account_category
    FROM `{project}.{dataset}.profit_and_loss_report_account_group`
    """
    df = client.query(query).to_dataframe()

    lookups = {
        'group': defaultdict(list),
        'sub_category': defaultdict(list),
        'category': defaultdict(list)
    }

    for _, row in df.iterrows():
        group = row['account_report_group']
        sub_category = row['account_report_sub_category']
        category = row['account_category']

        lookups['group'][group.lower()].append(group)
        lookups['sub_category'][sub_category.lower()].append(sub_category)
        lookups['category'][category.lower()].append(category)

    return lookups

lookups = create_lookup_tables()

memory = ConversationBufferMemory()
db = SQLDatabase.from_uri(sqlalchemy_url)
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory
)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=False,
    top_k=1000,
)

def find_matching_values(question, lookups):
    words = question.lower().split()
    matches = {
        'group': set(),
        'sub_category': set(),
        'category': set()
    }

    for word in words:
        for key in lookups:
            if word in lookups[key]:
                matches[key].update(lookups[key][word])

    return matches

def construct_filter_clause(matches):
    clauses = []
    for key, values in matches.items():
        if values:
            column = "account_report_group" if key == "group" else f"account_{key}"
            quoted_values = ["'{0}'".format(v) for v in values]
            clause = "{0} IN ({1})".format(column, ", ".join(quoted_values))
            clauses.append(clause)

    return " AND ".join(clauses) if clauses else ""

def determine_view(matches):
    if matches['group']:
        return "profit_and_loss_report_account_group_xa"
    elif matches['sub_category']:
        return "profit_and_loss_report_sub_categories_xa"
    else:
        return "profit_and_loss_report_categories_xa"

def extract_time_periods(question):
    """Extract time periods mentioned in the question."""
    # Patterns for various time formats
    year_pattern = r'\b(20\d{2})\b'
    month_year_pattern = r'\b(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+(\d{4})\b'
    quarter_pattern = r'\b(Q[1-4])\s+(\d{4})\b'

    time_periods = []

    # Extract years
    years = re.findall(year_pattern, question)
    for year in years:
        time_periods.append(('year', int(year)))

    # Extract month-year combinations
    month_years = re.findall(month_year_pattern, question, re.IGNORECASE)
    for month, year in month_years:
        date = parser.parse(f"{month} {year}")
        time_periods.append(('month', date))

    # Extract quarters
    quarters = re.findall(quarter_pattern, question)
    for quarter, year in quarters:
        quarter_num = int(quarter[1])
        start_month = (quarter_num - 1) * 3 + 1
        time_periods.append(('quarter', parser.parse(f"{year}-{start_month:02d}-01")))

    return time_periods

def construct_date_filter(time_periods):
    """Construct a SQL date filter based on extracted time periods."""
    filters = []
    for period_type, date in time_periods:
        if period_type == 'year':
            filters.append(f"EXTRACT(YEAR FROM date_month) = {date}")
        elif period_type == 'month':
            filters.append(f"date_month = DATE('{date.strftime('%Y-%m-%d')}')")
        elif period_type == 'quarter':
            quarter_end = date + relativedelta(months=2, day=31)
            filters.append(f"date_month BETWEEN DATE('{date.strftime('%Y-%m-%d')}') AND DATE('{quarter_end.strftime('%Y-%m-%d')}')")

    return " OR ".join(filters) if filters else ""

def strip_html(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text()

def summarize_content(question, content):
    prompt = PromptTemplate(
        input_variables=["question", "content"],
        template="""Given the following question and content, provide a concise summary of the content that is directly relevant to answering the question.
        Ignore any information that doesn't pertain to the question. All amounts should be stated in GBP (£).

        Question: {question}

        Content: {content}

        Relevant Summary:"""
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    summary = chain.run(question=question, content=content)
    return summary.strip()

def should_query_vector_store(question):
    prompt = PromptTemplate(
        input_variables=["question", "content_description", "valid_time_range"],
        template="""Given the following question and description of the content in a vector store,
        determine if the vector store is likely to contain information that can answer the question.
        Consider the following:
        1. Does the question ask about any of the key metrics or analyses mentioned in the content description?
        2. Does the question fall within the time frame covered by the vector store? Valid time range: {valid_time_range}
        3. Is the level of detail requested (category, subcategory, account group) available in the vector store?

        Respond with 'Yes' if the vector store is likely to contain relevant information, or 'No' if it's unlikely or unclear.

        Question: {question}

        Vector Store Content Description:
        {content_description}

        Decision (Yes/No):
        Explanation:"""
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(question=question,
                         content_description=vector_store_content_description,
                         valid_time_range=f"{valid_time_range[0].strftime('%B %Y')} to {valid_time_range[1].strftime('%B %Y')}")


    # Extract the decision from the response
    decision_match = re.search(r'Decision \(Yes/No\):\s*(Yes|No)', response, re.IGNORECASE)
    if decision_match:
        decision = decision_match.group(1).lower()
        return decision == 'yes'
    else:
        # If no clear decision is found in the expected format, look for a 'Yes' at the beginning of the response
        if response.strip().lower().startswith('yes'):
            return True
        else:
            return False

def evaluate_answer_relevance(question, answer):
    prompt = PromptTemplate(
        input_variables=["question", "answer"],
        template="""Given the following question and answer, evaluate how well the answer addresses the question.
        Provide a relevance score as a percentage and a brief explanation.

        Question: {question}

        Answer: {answer}

        Relevance Score (0-100%):
        Explanation:"""
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(question=question, answer=answer)
    return response

def ask_question(question, context=""):
    time_periods = extract_time_periods(question)

    if should_query_vector_store(question):
        month = time_periods[0][1] if time_periods and time_periods[0][0] in ['month', 'quarter'] else None
        filter_dict = {"month": month.strftime('%Y-%m-%d') if month else None}
        relevant_docs = vector_store.similarity_search(question, k=1, filter=filter_dict)

        if relevant_docs:
            pre_created_answer = relevant_docs[0].page_content
            stripped_answer = strip_html(pre_created_answer)
            summarized_answer = summarize_content(question, stripped_answer)
            answer = f"Based on the pre-created analysis: {summarized_answer}"
        else:
            answer = "No relevant pre-created analysis found. Falling back to SQL query."
    else:
        answer = "The question cannot be answered by pre-created analysis. Using SQL query."

    if "Using SQL query" in answer:
        matches = find_matching_values(question, lookups)
        filter_clause = construct_filter_clause(matches)
        view_name = determine_view(matches)

        date_filter = construct_date_filter(time_periods)

        if date_filter:
            filter_clause = f"({date_filter}) AND {filter_clause}" if filter_clause else date_filter

        instruction = f"""You are a knowledgeable finance data analyst working for Rittman Analytics.
        Use the `{project}.{dataset}.{view_name}` view to answer this question.
        Use the following SQL filter clause in your query: {filter_clause}
        When calculating revenue or any other financial metrics, make sure to aggregate the values for the entire time period(s) mentioned in the question (which may be months, quarters, or years).
        If multiple time periods are mentioned, provide a comparison between them.
        Please construct and execute a SQL query to answer the question, making sure to include the filter clause.
        Do not include markdown-style triple backticks in the SQL you generate and try to use or validate.

        {context}

        Question is: {question}
        """

        answer = agent_executor.run(instruction)

    relevance_evaluation = evaluate_answer_relevance(question, answer)

    return f"{answer}\n\nRelevance Evaluation:\n{relevance_evaluation}"

def extract_dates_from_question(question):
    """Extract all years mentioned in the question."""
    year_pattern = r'\b(20\d{2})\b'
    return list(set(re.findall(year_pattern, question)))

def get_similar_qa(question: str, k: int = 3):
    """Retrieve similar Q&A pairs from the vector store."""
    similar_qa = qa_vector_store.similarity_search(question, k=k)

    # Convert the results to the expected format
    converted_results = []
    for i, item in enumerate(similar_qa):
        if isinstance(item, Document):
            # If the item is already a Document, use it as is
            converted_results.append(item)
            print(f"Document {i+1} (already Document):")
            print(f"  Page content: {item.page_content[:100]}...")
            print(f"  Metadata: {item.metadata}")
        else:
            # If the item is a dict or has a different structure, create a new Document
            text = item.get('text', '')  # Use 'text' instead of 'content'
            metadata = {key: value for key, value in item.items() if key != 'text'}
            doc = Document(page_content=text, metadata=metadata)
            converted_results.append(doc)
            print(f"Document {i+1} (converted to Document):")
            print(f"  Page content: {text[:100]}...")
            print(f"  Metadata: {metadata}")

    return converted_results

def format_similar_qa(similar_qa):
    """Format similar Q&A pairs for inclusion in the prompt."""
    formatted = "Similar Q&A pairs from past interactions:\n\n"
    for i, doc in enumerate(similar_qa, 1):
        formatted += f"{i}. {doc.page_content}\n\n"
    return formatted

def ask_question_with_feedback_and_learning(question: str, max_iterations: int = 3) -> str:
    iteration = 0
    while iteration < max_iterations:
        # Retrieve similar Q&A pairs
        similar_qa = get_similar_qa(question)

        # Prepare the context with similar Q&A pairs
        context = format_similar_qa(similar_qa)

        # Use the modified ask_question function with context
        answer = ask_question(question, context)
        print(f"\nAnswer (Iteration {iteration + 1}):\n{answer}")

        user_satisfied = input("\nDid this answer your question sufficiently? (yes/no): ").lower().strip()

        if user_satisfied == 'yes':
            # Store the successful Q&A pair
            store_successful_qa(question, answer)
            return answer

        feedback = input("Please provide feedback on how the answer could be improved: ")

        # Use LLM to analyze feedback and improve the question
        improve_prompt = PromptTemplate(
            input_variables=["original_question", "answer", "feedback", "context"],
            template="""Given the original question, the provided answer, user feedback, and similar Q&A pairs from past interactions,
            please suggest an improved version of the question that addresses the user's concerns.

            Original Question: {original_question}

            Provided Answer: {answer}

            User Feedback: {feedback}

            {context}

            Improved Question:"""
        )

        improve_chain = LLMChain(llm=llm, prompt=improve_prompt)
        improved_question = improve_chain.run(
            original_question=question,
            answer=answer,
            feedback=feedback,
            context=context
        )

        print(f"\nImproved question based on feedback: {improved_question}")

        question = improved_question  # Update the question for the next iteration
        iteration += 1

    return "I apologize, but I couldn't provide a satisfactory answer within the maximum number of iterations. Please try rephrasing your question or contact support for further assistance."


def store_successful_qa(question: str, answer: str):
    """Store a successful question-answer pair in the vector store."""
    qa_pair = f"Q: {question}\nA: {answer}"

    # Generate embedding for the QA pair
    embedded_vector = embedding_model.embed_query(qa_pair)

    # Prepare the row to be inserted
    row = {
        'text': qa_pair,
        'embedding': embedded_vector,
        'id': str(uuid.uuid4())  # Generate a unique ID
    }

    # Insert the row into BigQuery
    client = bigquery.Client()
    table_id = f"{project}.{dataset}.successful_qa_pairs"

    errors = client.insert_rows_json(table_id, [row])

    if errors == []:
        print("Successful Q&A pair stored in vector store.")
    else:
        print(f"Errors occurred while storing Q&A pair: {errors}")



def main(reload_vector_storage=False):
    global valid_time_range, vector_store_content_description

    if reload_vector_storage:
        load_vector_storage()

    available_months = get_available_months()
    valid_time_range = get_valid_time_range(available_months)

    # Update the vector_store_content_description with the actual time range
    vector_store_content_description += f"""
    The analysis covers the period from {valid_time_range[0].strftime('%B %Y')} to {valid_time_range[1].strftime('%B %Y')}.
    For each month in this range, the analysis includes data for that month, comparisons to the two previous months, and year-to-date figures.
    """

    print("Hi! Ask me a question about our company's profit and loss data")
    while True:
        question = input("\nYour question (or type 'QUIT' to exit): ")
        if question.lower() == 'quit':
            break

        final_answer = ask_question_with_feedback_and_learning(question)
        print(f"\nFinal Answer: {final_answer}")
        print("\n---")

if __name__ == "__main__":
    main(reload_vector_storage=False)  # Set to True to reload vector storage